In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

import pandas as pd

options = Options()
options.headless = True

driver = webdriver.Chrome(options=options, service=ChromeService(
    ChromeDriverManager().install()))


In [2]:
url = 'https://www.transfermarkt.com/weltmeisterschaft-2022/gesamtspielplan/pokalwettbewerb/WM22/saison_id/2021'


In [4]:
driver.get(url)
nt = driver.find_elements(
    By.CSS_SELECTOR, '.large-6.columns .grid-view table.items tbody tr')


In [5]:
nt_list = []
for i in range(len(nt)):
    nt_name = nt[i].find_element(
        By.CSS_SELECTOR, 'td:nth-child(3) a:nth-child(1)').get_attribute('title')
    nt_url = nt[i].find_element(
        By.CSS_SELECTOR, 'td:nth-child(3) a:nth-child(1)').get_attribute('href')
    flag_url = nt[i].find_element(
        By.CSS_SELECTOR, 'td:nth-child(2) .flaggenrahmen').get_attribute('href')
    squad_url = f"{nt_url.replace('spielplan','kader')}/plus/1"
    nt_list.append({
        'nt_name': nt_name,
        'nt_url': nt_url,
        'flag_url': flag_url,
        'squad_url': squad_url,
    })
nt_df = pd.DataFrame(nt_list)
nt_df.to_excel('../../../data/wc22_team.xlsx')


In [2]:
def get_wc22_squad(driver, url):
    driver.get(url)
    players = driver.find_elements(By.CSS_SELECTOR, '#yw1 .items > tbody > tr')
    country = driver.find_element(
        By.CSS_SELECTOR, '.data-header__headline-container').text
    player_list = []
    print(f"Get {country} squad :")
    for i in range(len(players)):
        name = players[i].find_element(
            By.CSS_SELECTOR, 'td:nth-child(2) table img').get_attribute('title')
        src = players[i].find_element(
            By.CSS_SELECTOR, 'td:nth-child(2) table img').get_attribute('data-src')
        dob = players[i].find_element(
            By.CSS_SELECTOR, 'td:nth-child(3)').get_attribute('innerHTML')
        club = players[i].find_element(
            By.CSS_SELECTOR, 'td:nth-child(4) a').get_attribute('title')
        height = players[i].find_element(
            By.CSS_SELECTOR, 'td:nth-child(5)').get_attribute('innerHTML')
        caps = players[i].find_element(
            By.CSS_SELECTOR, 'td:nth-child(7)').get_attribute('innerHTML')
        market_value = players[i].find_element(
            By.CSS_SELECTOR, 'td:nth-child(10)').get_attribute('innerHTML')
        position = players[i].find_element(
            By.CSS_SELECTOR, 'td:nth-child(2) tbody tr:nth-child(2) td').get_attribute('innerHTML').strip()
        print(f"{i+1}. {name} - {position}")
        player_list.append({
            'name': name,
            'position': position,
            'dob': dob,
            'club': club,
            'caps': caps,
            'height': height,
            'market_value': market_value,
            'country': country
        })
    player_df = pd.DataFrame(player_list).reset_index(drop=True)
    return player_df


In [11]:
nt_df = pd.read_excel('../../../data/wc22_team.xlsx',index_col=0)
li = list(nt_df['squad_url'])

sq_df = pd.DataFrame()
for i in range(len(li)):
    d = get_wc22_squad(driver,li[i])
    sq_df = pd.concat([sq_df,d]).reset_index(drop=True)

Get Netherlands squad :
1. Justin Bijlow - Goalkeeper
2. Andries Noppert - Goalkeeper
3. Remko Pasveer - Goalkeeper
4. Matthijs de Ligt - Centre-Back
5. Virgil van Dijk - Centre-Back
6. Jurrien Timber - Centre-Back
7. Nathan Aké - Centre-Back
8. Stefan de Vrij - Centre-Back
9. Tyrell Malacia - Left-Back
10. Daley Blind - Left-Back
11. Jeremie Frimpong - Right-Back
12. Teun Koopmeiners - Defensive Midfield
13. Marten de Roon - Defensive Midfield
14. Frenkie de Jong - Central Midfield
15. Kenneth Taylor - Central Midfield
16. Denzel Dumfries - Right Midfield
17. Xavi Simons - Attacking Midfield
18. Steven Berghuis - Attacking Midfield
19. Davy Klaassen - Attacking Midfield
20. Cody Gakpo - Left Winger
21. Steven Bergwijn - Left Winger
22. Noa Lang - Left Winger
23. Memphis Depay - Centre-Forward
24. Wout Weghorst - Centre-Forward
25. Vincent Janssen - Centre-Forward
26. Luuk de Jong - Centre-Forward
Get Senegal squad :
1. Edouard Mendy - Goalkeeper
2. Alfred Gomis - Goalkeeper
3. Seny Di

In [ ]:
sq_df.to_excel("../../../data/wc22_squad.xlsx")

In [4]:
sq_df = pd.read_excel("../../../data/wc22_squad.xlsx",index_col=0)

In [ ]:
def format_market_value(mv):
    if mv[-1] == 'm':
        val = int(float(mv[:-1])*1000000)
    elif mv[-1] == 'k':
        val = int(float(mv[:-1])*1000)
    else:
        val = mv
    return val


In [ ]:
sq_df['market_value'] = sq_df['market_value'].map(lambda x:format_market_value(x))

In [10]:
sq_df.groupby('country')['name'].count().reset_index(name='count').sort_values(['count'],ascending=False)

,country,count
16,Mexico,32
28,Tunisia,30
21,Qatar,30
9,Ecuador,28
22,Saudi Arabia,28
24,Serbia,27
1,Australia,26
30,Uruguay,26
29,United States,26
27,Switzerland,26
